In [4]:
#import libraries

import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
#import training data

df = pd.read_csv("C:/Users/270168/Downloads/personal/nlp-starter-test/social_media_clean_text.csv")

In [3]:
#check the data 

df.head()

,text,choose_one,class_label
0,just happened a terrible car crash,Relevant,1
1,our deeds are the reason of this earthquake m...,Relevant,1
2,"heard about earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,forest fire near la ronge sask canada,Relevant,1


In [13]:
#separate text column for preprocessing

df["processed_text"] = df.iloc[:,0]

In [21]:
def process (G):
    tok = G.split(" ")
    lower_tok = [t.upper() for t in tok]
    return lower_tok

In [22]:
df["processed_text"] = df["processed_text"].apply(process)